# What happens:
File is created and email is prepared

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import datetime
import win32com.client as win32
import calendar
from openpyxl import Workbook, load_workbook
from IPython.lib.display import Audio

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [2]:
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports/Guarantees Report"
os.chdir(directory)

rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0002"

Please enter the RD (YYYYMMDD): 20221031


### Make connection with Oracle DB

In [3]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

### Check if WS are completed

In [9]:
%%time
d1 = pd.read_sql_query("""select entity.entity_desc as guarantor_name, guarantee.*
from guarantee
left join entity on guarantee.issuer_guarantor = entity.entity_code
where guarantee.dim1 is not null""", connection_RAY)

Wall time: 153 ms


In [10]:
d1

,guarantor_name,partition_key,contract_reference,contract_desc,book_code,contract_type,collateral_mode,issuer_guarantor,intermediate_guarantor,beneficiary,...,check_error_status_o,check_error_status_t,check_error_status_w,check_error_status_y,check_error_status_d,check_error_status_v,check_error_status_x,check_error_status_z,check_error_status_f,check_error_status_e
0,Fond Rozvytku Pidpryiemnytstva,202210310000,GUA_BDF_COV_AVAL_2020,None,AVAL_B,GUA,M,1856916,None,None,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# determining the name of the file
file_name = 'Covid 19 guarantees in RAY ' + rd + '.xlsx'

# saving the excel
d1.to_excel(file_name, index=False)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [7]:
#get last month
dt = datetime.datetime.today()
month_name = calendar.month_name[dt.month-1].upper()
year  = dt.year

In [8]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'brigitte.aigner@rbinternational.com; michael.bruckner@rbinternational.com'
mail.Cc = 'zoltan.szebenyi@rbinternational.com; nertila.gjakova@rbinternational.com; schlemoun.piro@rbinternational.com'
mail.Subject = 'RBI Covid guarantees as of ' + rd + ' - RAY Report'
mail.Body = '''Dear Brigitte,

Kindly find attached the Covid 19 guarantees Report as of ''' + month_name + '''

Kr
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''

# To attach a file to the email (optional):
attachment  = directory + "/" + file_name
mail.Attachments.Add(attachment)

mail.Display()